In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# MOdel selection
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LogisticRegression

# Model hyper parameter tuning
from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_auc_score,roc_curve

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import os


import warnings  
warnings.filterwarnings('ignore')
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import NearestNeighbors

In [18]:
def read_file(folder, samp = False, lof=False):
    df = pd.read_csv(folder)
    if samp==True:
        df = df.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
    if lof==True:
        #df_new = pd.DataFrame(scaler.fit_transform(df_new))
        df = filter_lof(df)
        #df_new = scaler.inverse_transform(df_new)
    return df

In [40]:
def filter_lof(df, k=20) :
    lof = LocalOutlierFactor(n_neighbors=k)
    df2 = pd.DataFrame.copy(df)
    df2["_lof"] = lof.fit_predict(df2)
    return df2[df2["_lof"]>0].drop(columns="_lof").reset_index(drop=True)

In [42]:
def filter_lof(df, k=20) :
    lof = LocalOutlierFactor(n_neighbors=k)
    scaler = StandardScaler()
    df2 = pd.DataFrame.copy(df)
    df2 = df2.drop(columns = ['churn'])
    df2 = pd.DataFrame(scaler.fit_transform(df2))
    df2["_lof"] = lof.fit_predict(df2)
    df2 = df2[df2["_lof"]>0].drop(columns="_lof")#.reset_index(drop=True)
    df2 = pd.DataFrame(scaler.inverse_transform(df2))
    df2['churn'] = df['churn']
    return df2.reset_index(drop=True)

In [31]:
def crop_attila(df, cluster_size):
    firsts = []
    files = df['churn'].unique()
    to_delete_list = []
    for i in range(len(files)):
        #last type
        if i==len(files)-1:
            first = len(df)
            firsts.append(first)
        else:
            first = df.index[df['churn']==files[i+1]].tolist()[0]
            firsts.append(first)
        #first type
        if i==0:
            to_delete = first%cluster_size
        else:
            to_delete = (first-firsts[i-1])%cluster_size
        if cluster_size == 3:
            to_delete = to_delete-3
        #chosing rows to drop
        for j in range(to_delete):
            to_delete_list.append(first-j-1)
    df = df.drop(to_delete_list)
    df = df.reset_index(drop=True)
    return df

In [ ]:
df = pd.read_csv('/data/dataprivacy/churn-anonymized/v1/alex2/k10_e10_lof/telecom_churn_data_pre_nodates.csv')

In [ ]:
len(df[df['churn']<0.5])/len(df)

In [ ]:
data = pd.read_csv('/data/dataprivacy/churn/telecom_churn_data_pre_nodates.csv')

In [ ]:
len(data[data['churn']<0.5])/len(data)

In [21]:
def is_churned(_x):
    if ((_x.total_ic_mou_9 == 0) & (_x.total_og_mou_9 == 0) & (_x.vol_2g_mb_9 == 0) & (_x.vol_3g_mb_9 == 0)):
        return 1
    else:
        return 0

In [ ]:
data_churn = df.drop(['churn'],axis = 1)
data_churn['churn'] = data_churn.apply(is_churned, axis=1)

In [51]:
def eval_privacy(algos):
    df_avg_ident = pd.DataFrame(columns = ['avg_per_10'])
    for algo in algos:
        print(algo)
        if algo=='attila1' or algo=='attila2':
            df_original = read_file('/data/dataprivacy/churn/telecom_churn_data_pre_nodates.csv', samp=False, lof=False)
            print('attila')
        else:
            df_original = read_file('/data/dataprivacy/churn/telecom_churn_data_pre_nodates.csv', samp=False, lof=True)
            print('nemattila')
        for file in os.listdir('/data/dataprivacy/churn-anonymized/v1/'+algo):
            if file.startswith('c') or file.startswith('k'):
                print(file)
                if algo=='attila1' or algo=='attila2':
                    if int(file[1])==3 or int(file[1])==5:
                        num = int(file[1:2])
                    else:
                        num = int(file[1:3])
                    print(num)
                    df_original_cropped = crop_attila(df_original, num)
                    df = read_file('/data/dataprivacy/churn-anonymized/v1/'+algo+'/'+file+'/telecom_churn_data_pre_nodates.csv')
                    print(len(df_original_cropped))
                    print(len(df))
                    X = df_original_cropped.iloc[:,:-1]
                else:
                    df = read_file('/data/dataprivacy/churn-anonymized/v1/'+algo+'/'+file+'/telecom_churn_data_pre_nodates.csv')
                    print(len(df_original))
                    print(len(df))              
                    X = df_original.iloc[:,:-1]
                
                nbrs_original = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(X)
                distances_original, indices_original = nbrs_original.kneighbors(X)
                indices_original = indices_original[:,1:]
                inds = pd.isnull(df).any(1).to_numpy().nonzero()
                df = df.dropna().reset_index(drop=True)
                X = X.drop(inds[0]).reset_index(drop=True)
                Y = df.iloc[:,:-1]
                nbrs = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(Y)
                distances, indices = nbrs.kneighbors(Y)
                indices = indices[:,1:]
                identical = []
                for l in range(len(indices)):
                    identical.append(len(set(indices_original[l]).intersection(indices[l])))
                df_avg_ident.loc[algo+'_'+file] = sum(identical)/len(identical)
                df_avg_ident.to_csv('churn_eval_privacy_v1.csv')

In [ ]:
algos = ['attila1','attila2','alex1b','alex2']
eval_privacy(algos)

attila1
attila
c3_e1
3
99999
99999
c20_e10
20
99980
99980
c10_e1
10
99990
99990
c3_e10
3
99999
99999
c10_e10
10
99990
99990
c5_e10
5
99995
99995
c5_e1
5
99995
99995
c20_e1
20
99980
99980
attila2
attila
c3_e1
3
99999
99999
c20_e10
20
99980
99980
c10_e1
10
99990
99990


In [ ]:
df_eval = pd.DataFrame(columns = ['original','anonymized-churn','churn-anonymized'])
for algo_anon in ['attila1']:#, 'attila2', 'alex1b', 'alex2']:
    print(algo_anon)
    if algo_anon=='alex1b' or 'alex2':
        df_original = read_file('/data/dataprivacy/churn/telecom_churn_data_pre_nodates.csv', lof=True)
        df_original = df_original.dropna().reset_index(drop=True)
    if algo_anon=='attila1' or 'attila2':
        df_original = read_file('/data/dataprivacy/churn/telecom_churn_data_pre_nodates.csv')
        if int(file[1])==3 or int(file[1])==5:
            num = int(file[1:2])
        else:
            num = int(file[1:3])
        df_original = crop_attila(df_original, num)
    print(len(df_original))
    for file in os.listdir('/data/dataprivacy/churn-anonymized/v1/'+algo_anon):
        print(file)
        df = read_file('/data/dataprivacy/churn-anonymized/v1/'+algo_anon+'/'+file+'/'+'telecom_churn_data_pre_nodates.csv')
        print(len(df))
        similar1=0
        similar2=0
        
        #df_eval.loc[algo_anon+'_'+file] = 
        #df_eval.to_csv('churn_data.csv')